# Revisão

In [2]:
!pip install plotly=='4.8.1'

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [4]:
king = pd.read_csv("https://raw.githubusercontent.com/vnikoofard/python_online_2020/master/data/kc_house_data.csv")

In [5]:
king.date = pd.to_datetime(king.date, infer_datetime_format=True)
king.yr_built = pd.to_datetime(king.yr_built,format='%Y')

king['age'] = king.date.dt.year - king.yr_built.dt.year

king.age = pd.to_numeric(king.age)

In [6]:
df = king.drop(['id', 'date', 'zipcode', 'yr_renovated', 'yr_built'], axis=1)

In [7]:
correlation = df.corr()

cols = correlation['price'][(abs(correlation['price']>0.3))]
l = cols.index.to_list()

important = df[l]

In [8]:
important = important[important.sqft_living<5230]

In [9]:
important = important[important.bedrooms<11]


In [10]:
important = important[important.sqft_basement<1740]

In [11]:
important = important[important.sqft_above<4400]

In [12]:
important.shape

(21216, 10)

In [13]:
df.shape

(21613, 17)

In [14]:
important.head()

,price,bedrooms,bathrooms,sqft_living,view,grade,sqft_above,sqft_basement,lat,sqft_living15
0,221900.0,3,1.00,1180,0,7,1180,0,47.5112,1340
1,538000.0,3,2.25,2570,0,7,2170,400,47.7210,1690
2,180000.0,2,1.00,770,0,6,770,0,47.7379,2720
3,604000.0,4,3.00,1960,0,7,1050,910,47.5208,1360
4,510000.0,3,2.00,1680,0,8,1680,0,47.6168,1800


In [15]:
X = important.drop('price', axis = 1)
y = important['price']

In [16]:
X.shape

(21216, 9)

In [17]:
def norm(x): 
  return (x - x.mean(axis=0))/x.std(axis=0)

In [18]:
X_norm = norm(X)

In [19]:
X_norm

,bedrooms,bathrooms,sqft_living,view,grade,sqft_above,sqft_basement,lat,sqft_living15
0,-0.390501,-1.485038,-1.045389,-0.294231,-0.545522,-0.754613,-0.672624,-0.347763,-0.954146
1,-0.390501,0.227033,0.680221,-0.294231,-0.545522,0.563896,0.304055,1.160677,-0.414070
2,-1.513721,-1.485038,-1.554382,-0.294231,-1.447330,-1.300663,-0.672624,1.282186,1.175294
3,0.732719,1.254276,-0.077061,-0.294231,-0.545522,-0.927751,1.549321,-0.278740,-0.923284
4,-0.390501,-0.115381,-0.424666,-0.294231,0.356286,-0.088700,-0.672624,0.411490,-0.244332
...,...,...,...,...,...,...,...,...,...
21608,-0.390501,0.569448,-0.610883,-0.294231,0.356286,-0.288474,-0.672624,1.004656,-0.660962
21609,0.732719,0.569448,0.357445,-0.294231,0.356286,0.750352,-0.672624,-0.351358,-0.198040
21610,-1.513721,-1.827452,-1.244020,-0.294231,-0.545522,-0.967706,-0.672624,0.250436,-1.447929
21611,-0.390501,0.569448,-0.523982,-0.294231,0.356286,-0.195246,-0.672624,-0.180238,-0.846131


In [20]:
X_norm.mean()

bedrooms         9.359762e-16
bathrooms       -8.026500e-15
sqft_living      2.806955e-17
view             2.690103e-16
grade           -7.106866e-15
sqft_above       1.431102e-15
sqft_basement   -8.418459e-16
lat              2.871893e-14
sqft_living15   -6.524444e-17
dtype: float64

Regressão Linear

In [21]:
a,b,c,d = (1,2,3,4)

In [22]:
b

2

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)

In [24]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [25]:
y_pred = lr.predict(X_test)

In [26]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_pred, y_test))

In [27]:
rmse

185817.64127795477

In [28]:
important.price.describe()

count    2.121600e+04
mean     5.187656e+05
std      3.048539e+05
min      7.500000e+04
25%      3.200000e+05
50%      4.470000e+05
75%      6.300000e+05
max      3.650000e+06
Name: price, dtype: float64

Regressão Polinomial

In [38]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(4)

xtrain_poly = poly.fit_transform(X_train)
xtest_poly = poly.fit_transform(X_test)

In [39]:
xtrain_poly.shape

(16972, 715)

In [40]:
X_train.shape

(16972, 9)

In [41]:
lr_poly = LinearRegression()
lr_poly.fit(xtrain_poly, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [42]:
y_pred_poly = lr_poly.predict(xtest_poly)

In [43]:
from sklearn.metrics import mean_squared_error
rmse_poly = np.sqrt(mean_squared_error(y_pred_poly, y_test))

In [44]:
rmse_poly

153469.5255451676

In [45]:
rmse

185817.64127795477

In [37]:
fig = go.Figure()
fig.add_scatter(y = y_pred_poly[:24], name = 'y_pred_poly')
fig.add_scatter(y = y_test[:24], name = 'y_test')

Arvore de Decisão

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor()

tree.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [48]:
y_pred_tree = tree.predict(X_test)

In [49]:
from sklearn.metrics import mean_squared_error
rmse_tree = np.sqrt(mean_squared_error(y_pred_tree, y_test))

In [50]:
rmse_tree

185115.03522992737

In [52]:
fig = go.Figure()
fig.add_scatter(y = y_pred_poly[:24], name = 'y_pred_poly')
fig.add_scatter(y = y_pred_tree[:24], name = 'y_pred_tree')

fig.add_scatter(y = y_test[:24], name = 'y_test')

Random Forest

In [60]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators = 400, verbose=2)

rfr.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building tree 42 of 400
b

[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   35.0s finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=None, verbose=2, warm_start=False)

In [61]:
y_pred_rfr = rfr.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    0.6s finished


In [62]:
from sklearn.metrics import mean_squared_error
rmse_rfr = np.sqrt(mean_squared_error(y_pred_rfr, y_test))

In [63]:
rmse_rfr

138373.47273120293

In [59]:
fig = go.Figure()
fig.add_scatter(y = y_pred_rfr[:24], name = 'y_pred_rfr')

fig.add_scatter(y = y_test[:24], name = 'y_test')

XGBOOST


In [71]:
from sklearn import ensemble

gbr = ensemble.GradientBoostingRegressor(n_estimators = 300, verbose=2)

gbr.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1 83439523811.9527            8.63s
         2 74955081285.3530            7.62s
         3 67732031136.3614            7.26s
         4 61674914820.7551            7.04s
         5 56605198947.2119            6.93s
         6 52310665689.8888            6.84s
         7 48576253226.1291            6.76s
         8 45492531448.7532            6.70s
         9 42792639753.0246            6.74s
        10 40448574838.2782            6.82s
        11 38371915648.1438            6.76s
        12 36607372005.5549            6.69s
        13 35127454958.8643            6.64s
        14 33731265350.8624            6.60s
        15 32543822974.7870            6.60s
        16 31544294014.4409            6.56s
        17 30591885401.5517            6.51s
        18 29808994737.7042            6.52s
        19 29057607299.2096            6.49s
        20 28308819697.8515            6.46s
        21 27749151685.0586            6.42s
        2

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=300,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=2, warm_start=False)

In [73]:
y_pred_gbr = gbr.predict(X_test)

In [74]:
from sklearn.metrics import mean_squared_error
rmse_gbr = np.sqrt(mean_squared_error(y_pred_gbr, y_test))

In [75]:
rmse_gbr

139925.4723651188

Save model

In [64]:
import joblib

In [65]:
joblib.dump(rfr, 'model.pkl')

['model.pkl']

In [66]:
rfr_new = joblib.load('model.pkl')

In [67]:
important.to_csv('king_important.csv')

In [ ]:
rfr_new.predict